In [ ]:
import sys

# --- CONFIGURATION ---
OUTPUT_FILE = "solute.inp"
ROCK_NAME   = "Backg"   # Must match your flow.inp rock name exactly

# Simulation Settings
# ISPIA=2 (Iterative solver), INITS=0 (Read concentrations)
SL_FLAGS = "200002001100" 

# Output Control
# MINFLAG=1 (Print Vol Fraction change), GFLAG=1 (Gas output)
OUT_FLAGS = "11000000"

# Coupling
# KCPL=1 (Feedback enabled: Permeability changes as minerals precipitate)
KCPL = 1

# Diffusion coeffs (Standard aqueous species and gas)
DIFF_AQ = 1.0e-9
DIFF_GAS = 1.0e-5

print(f"--- Generating {OUTPUT_FILE} ---")

with open(OUTPUT_FILE, 'w') as f:
    # 1. TITLE
    f.write("Solute Transport Input for Carbon Mineralization\n")

    # 2. GLOBAL FLAGS (Record 2)
    # ISPIA, ITRMAX, etc.
    # Format: I1, I1, I2, I2, 5X, I5, E10.3 ... difficult fixed format.
    # We use a robust standard string for ECO2N
    # KCPL is the critical one at column 25.
    f.write(f"{SL_FLAGS}      {KCPL}   0.5000E+00 0.1000E-05 0.6000E+00\n")

    # 3. OUTPUT & WEIGHTING (Record 3)
    f.write("   1.00000   1.00000\n") 
    
    # 4. VELOCITY & DIFFUSION (Record 4)
    # VCON, D0(aq), D0(gas)
    f.write(f"{' ':<10}{DIFF_AQ:10.3E}{DIFF_GAS:10.3E}\n")

    # 5. DATA PRINTOUT (Record 6 & 7)
    f.write("    0    0    0    0\n") # KDATA
    f.write(f"{OUT_FLAGS}\n")          # MINFLAG, GFLAG, etc.

    # 6. NODES FOR HISTORY (Record 8)
    # We leave this blank or put a dummy because we rely on 3D output files usually.
    f.write("\n") 

    # 7. AQUEOUS SPECIES (Record 9)
    # These indices refer to the order in chemical.inp
    # Listing 1 to 10 is usually safe for standard outputs
    f.write("12345678901234567890\n") # Just a mask
    f.write("    1    2    3    4    5    6    7    8    9   10\n")

    # 8. MINERAL ZONES (Record 10)
    # "1" means we want output for mineral #1 (e.g. Calcite)
    f.write("    1    2    3    4    5\n")

    # 9. GAS ZONES (Record 11)
    f.write("    1\n")

    # 10. ADSORPTION ZONES (Record 12)
    f.write("    0\n")

    # 11. LINEAR ADSORPTION (Record 13)
    f.write("    0\n")

    # =======================================================
    # CRITICAL: MAPPING ROCKS TO ZONES (Record 14 & 15)
    # =======================================================
    # In 3D, we map by MATERIAL NAME, not by node numbers.
    # The format is specific: RockName (A5) -> ZoneID
    
    # Header for Record 14: Default Zones
    # IZMIDF (Min), IZGIDF (Gas), IZADDF (Ads), IZPPDF (Perm Law), IZOXDF
    # We set Default Mineral Zone = 1.
    # We set Permeability Law = 8 (Verma-Pruess) which is best for mineralization.
    f.write("    1    1    0    8    0\n")

    # Record 15: Specific Mappings
    # We explicitly map our "Backg" rock to Zone 1 and Perm Law 8.
    # Format: RockName(A5), 5X, IZMI, IZGI, IZAD, IZPP
    f.write(f"{ROCK_NAME:<5}{' ':<5}    1    1    0    8\n")
    
    # End of file
    f.write("end\n")

print("Done. Key configurations set:")
print(f" - Coupling (KCPL): {KCPL} (Porosity feedback ON)")
print(f" - Perm Law: 8 (Verma-Pruess for clogging)")
print(f" - Rock '{ROCK_NAME}' mapped to Mineral Zone 1")

--- Generating solute.inp ---
Done. Key configurations set:
 - Coupling (KCPL): 1 (Porosity feedback ON)
 - Perm Law: 8 (Verma-Pruess for clogging)
 - Rock 'Backg' mapped to Mineral Zone 1
